In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('DataSource/application_txn.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(5)

,userid,action,objecttype,objectid,ads_on_page,sponsorid,artistid,time_stamp2,exp,coin
0,59391fbfe4b0a0f774fe022e,VIEW,MISSIONPAGE,1494491526872/1498239229913/1496144767898/1498...,NaN,NaN,NaN,2017-07-01 00:04:22.842,0,0
1,592f8ddfe4b09c162d715525,VIEW,MISSIONPAGE,1498802539453/1498481977332/1498212249886/1498...,NaN,NaN,NaN,2017-07-01 00:35:59.743,0,0
2,592e1c65e4b09c162d70ac5b,VIEW,MISSIONPAGE,1494491526872/1498217706019/1496144767898/1498...,NaN,NaN,NaN,2017-07-01 00:21:00.408,0,0
3,5945552ee4b0db45c711c290,VIEW,MISSIONPAGE,1494491526872/1498239229913/1496144767898/1498...,NaN,NaN,NaN,2017-07-01 00:06:10.999,0,0
4,593e4481e4b06f7fcb9eabe4,VIEW,FEEDPAGE,1495095167612/59567fdae4b0a6ba1750b8e0/59567cc...,NaN,NaN,NaN,2017-07-01 00:32:49.475,0,0


In [4]:
record = df[['userid','objecttype','action','time_stamp2']].values
record[0,3][0:10]

'2017-07-01'

In [20]:
#Create dictionary for storing userId mapping to list of unique day that they particiated in application 
#For example {'59391fbfe4b0a0f774fe022e': [2017-07-01,2017-06-01]}
dictDAU = {}
for row in record:
    dateStamp = row[3][0:10]
    userId = row[0]
    
    if dictDAU.get(userId,0) == 0:
         dictDAU[userId] = [dateStamp]
    else:
        if dateStamp not in dictDAU[userId]:
            dictDAU[userId].append(dateStamp)
            
#Create frequency dictionary of DAU
#for example {25: ['592de8c1e4b09c162d7093af','592f9662e4b09c162d715ac3'], 24:[59567fd8e4b0f93e9119cbf1]}
freqDAU = {}
for key in dictDAU:
    freq = len(dictDAU[key])
           
    if freqDAU.get(freq,0) == 0:
        freqDAU[freq] = [key]
    else:
        freqDAU[freq].append(key)

#Find userIds who have the highest DAU
freqDAU = sorted(freqDAU.items(),reverse=True)
mostDAUUser = freqDAU[0][1]


'592de8c1e4b09c162d7093af'

In [26]:
#print(df)
#print(df.loc('592de8c1e4b09c162d7093af'))
mostDAUDf = df.loc[df['userid'].isin(mostDAUUser)]

mostDAUDf.groupby('action').size()


action
APPIN             10240
APPOUT             9406
BONUS               152
CLICK              8915
COMMENT              47
COMPLETE           1928
DELETECOMMENT         2
FAIL                736
FOLLOW              402
INCOMPLETE          358
JOIN               3862
LIKE              35032
LOGIN               194
REDEEM_REQUEST      284
SCORE_GAUGE       23552
SHARE              7263
UNFOLLOW             33
UNLIKE              271
VIEW              48793
dtype: int64

In [85]:
x = mostDAUDf.groupby(['objecttype','action']).size().unstack()
x
#mostActivityDAU = mostDAUDf['objecttype'].value_counts()
#mostDAUDf['objecttype'].value_counts().plot(kind='barh')
#plt.show()


action,BONUS,CLICK,COMMENT,COMPLETE,DELETECOMMENT,FAIL,FOLLOW,INCOMPLETE,JOIN,LIKE,REDEEM_REQUEST,SCORE_GAUGE,SHARE,UNFOLLOW,UNLIKE,VIEW
objecttype,,,,,,,,,,,,,,,,
ADSFEED,NaN,124.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,306.0,NaN,NaN,4.0,NaN,118.0,120.0
ADSVIDEO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,18194.0,NaN,NaN,NaN,18590.0
ARTIST,NaN,NaN,NaN,NaN,NaN,NaN,402.0,NaN,NaN,NaN,NaN,NaN,NaN,33.0,NaN,NaN
ARTIST_RANK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1057.0
ARTIST_WEEKLY_RANK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1192.0
EVENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN
EXCLUSIVE,NaN,NaN,7.0,8.0,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,14.0,NaN,NaN,167.0
EXCLUSIVEPAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,912.0
FEED,NaN,1440.0,20.0,NaN,1.0,NaN,NaN,NaN,NaN,32331.0,NaN,NaN,1322.0,NaN,125.0,NaN


In [7]:
df.describe()

,sponsorid,artistid,exp,coin
count,2.639460e+05,1.686690e+05,716814.000000,716814.000000
mean,1.493216e+12,1.479089e+12,6.997722,6.128121
std,4.551081e+09,1.627913e+10,24.383469,75.568945
min,1.477035e+12,1.460953e+12,0.000000,-4000.000000
25%,1.494388e+12,1.460953e+12,0.000000,0.000000
50%,1.494388e+12,1.478771e+12,0.000000,0.000000
75%,1.494388e+12,1.494819e+12,5.000000,0.000000
max,1.498211e+12,1.500368e+12,500.000000,10000.000000


In [8]:
df.groupby('userid').size()


userid
592da01fe4b0a4ee112b84fb     311
592da02be4b09c162d708589     854
592da035e4b0a4ee112b8505     795
592da070e4b09c162d70859a     626
592da096e4b0a4ee112b852a     469
592da0bae4b0a4ee112b8545     511
592da122e4b09c162d708602    1882
592da13ce4b09c162d70862a     598
592da3a2e4b0a4ee112b8618     236
592da3a3e4b09c162d708691    1097
592da691e4b0a4ee112b86bb    1858
592da6ace4b09c162d70873f    5367
592da6e4e4b09c162d708759    1015
592da826e4b09c162d7087b7     112
592dab2be4b09c162d708841     339
592dab88e4b0a4ee112b8872      37
592dac6ee4b09c162d7088c2     180
592dac7ee4b09c162d7088d9     405
592daf42e4b0a4ee112b8a3e    1381
592dafffe4b09c162d708a16       2
592db0e7e4b09c162d708a4d     855
592db0f5e4b0a4ee112b8abf    2093
592db1fce4b09c162d708a95    1009
592db2b9e4b0a4ee112b8b39    3331
592db383e4b09c162d708b97     870
592db4a3e4b0a4ee112b8cd9    1744
592db4bfe4b0a4ee112b8ceb     592
592db682e4b0a4ee112b8dca     221
592db6d6e4b0a4ee112b8ddc      83
592db807e4b09c162d708de7      15
   

In [9]:
df.isnull().sum()

userid              0
action              0
objecttype     116491
objectid       177525
ads_on_page    681422
sponsorid      452868
artistid       548145
time_stamp2         0
exp                 0
coin                0
dtype: int64

In [10]:
df.groupby(['userid','exp']).sum()


sponsorid      artistid  coin
userid                   exp                                  
592da01fe4b0a4ee112b84fb 0    1.553299e+14  2.089539e+13 -1988
                         5    2.988626e+12           NaN     0
                         9    2.992964e+12           NaN     0
                         41   2.992591e+12           NaN     0
                         42   2.988776e+12           NaN     0
                         46   2.955297e+12           NaN     0
                         47   2.988623e+12           NaN     0
                         48   8.897989e+12           NaN     0
                         200  2.988776e+12           NaN  2000
                         500  5.977553e+12           NaN  2000
592da02be4b09c162d708589 0    1.404725e+14  2.071125e+14 -1000
                         6             NaN  1.495013e+12     0
                         9             NaN  1.460953e+12     0
                         10            NaN  2.807186e+13     0
                         20            NaN  2.955773e+12     0
                         49   2.988625e+12           NaN     0
                         200  2.988776e+12           NaN  2000
                         500  2.988776e+12           NaN  1000
592da035e4b0a4ee112b8505 0    3.078105e+14  2.991245e+13  1002
                         5             NaN  1.494929e+12     0
                         41            NaN  1.492659e+12     0
                         44   2.955297e+12           NaN     0
                         46   2.988625e+12           NaN     0
                         50   1.793266e+13           NaN   600
                         100  5.977553e+12           NaN   400
                         200  2.988776e+12           NaN  2000
                         500  5.977553e+12           NaN  2000
592da070e4b09c162d70859a 0    1.763378e+14  6.368098e+13     0
592da096e4b0a4ee112b852a 0    1.046240e+14  9.460519e+13 -1000
                         10            NaN  5.950889e+12     0
...                                    ...           ...   ...
59774e18e4b09713a27f15bd 0    2.988776e+12  7.386823e+12     0
59774fb9e4b0a7427fcc7d26 0             NaN  7.358059e+12     0
5977559de4b0a7427fcc7e73 0    5.977553e+12  8.908975e+12     0
59775768e4b0d4cb4b85f1a2 0             NaN  7.389277e+12     0
                         7             NaN  1.494819e+12     0
5977593ce4b0fe8edbc2cb8d 0    9.862962e+13  6.213970e+13   600
                         5    5.977553e+12  5.945294e+12     0
                         6    8.966329e+12           NaN     0
                         7    1.494388e+13  4.450979e+12     0
                         8    5.977553e+12  5.993947e+12     0
                         9    8.966329e+12  2.939722e+12     0
                         200  2.988776e+12           NaN  2000
597759b7e4b09713a27f183a 0             NaN           NaN     0
59775a7ee4b0d4cb4b85f2bf 0    2.092143e+13  1.182906e+13     0
                         5             NaN  1.496232e+12     0
                         6             NaN  1.498472e+12     0
                         8             NaN  5.956368e+12     0
                         9             NaN  7.417321e+12     0
                         200  2.988776e+12           NaN  2000
597763e0e4b0fe8edbc2cdde 0             NaN  3.843928e+13     0
597767a6e4b0a7427fcc824e 0    2.988776e+12  7.378037e+12     0
597767b6e4b0d4cb4b85f56b 0             NaN  1.184163e+13   200
5977681fe4b0d4cb4b85f586 0             NaN  1.765852e+13     0
59776bdce4b0a7427fcc831a 0    5.977553e+12  7.385682e+12     0
59776e2ce4b09713a27f1c44 0    2.988761e+13  2.346228e+13   202
                         5             NaN  2.922100e+12     0
                         8             NaN  2.922272e+12     0
                         9             NaN  1.460953e+12     0
                         41   2.988623e+12           NaN     0
597772e1e4b0a7427fcc83d3 0    5.977553e+12  7.362001e+12     0

[15054 rows x 3 columns]

In [11]:
groupby_exp = df.groupby(['userid']).apply(lambda x: (x['exp']).sum())

In [12]:
groupby_exp = groupby_exp.sort_values()
top10_user = groupby_exp.tail(10)

In [13]:
df10 = pd.DataFrame(top10_user, columns=['exp'])


df10

,exp
userid,
5971b134e4b0a7427fcbd8f6,86330
592e0048e4b0a4ee112b9913,88730
59438762e4b0db45c7118987,92356
5944fa9ce4b0b1300401cb5a,96115
597200b2e4b0d4cb4b8557af,97005
592e18a9e4b09c162d70a9af,134077
592f9662e4b09c162d715ac3,161570
592e44f9e4b0a4ee112bcc68,239941
592dbb1ee4b09c162d708eb8,294173
